In [24]:
import pandas as pd
import nltk
import re
import os
from datetime import datetime
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
from nltk.collocations import TrigramAssocMeasures, TrigramCollocationFinder
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from collections import Counter
import csv

FOLDER_KAMUS_UTAMA = 'Kamus'
FOLDER_KAMUS_MENTAH = 'Kamus Mentah'
FOLDER_LOG_RIWAYAT = 'Riwayat'
FILE_LOG_RIWAYAT = os.path.join(FOLDER_LOG_RIWAYAT, 'riwayat_proyek.csv')

try:
    os.makedirs(FOLDER_KAMUS_MENTAH, exist_ok=True)
    print(f"✅ Folder output '{FOLDER_KAMUS_MENTAH}' telah disiapkan.")
except Exception as e:
    print(f"⚠️ Gagal membuat folder '{FOLDER_KAMUS_MENTAH}': {e}")

def log_event_ke_riwayat(kategori, keterangan):
    """Menulis satu baris log ke file riwayat proyek."""
    try:
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        data_row = [timestamp, kategori, keterangan]
        
        file_exists = os.path.isfile(FILE_LOG_RIWAYAT)
        with open(FILE_LOG_RIWAYAT, mode='a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            if not file_exists:
                writer.writerow(['timestamp', 'kategori_event', 'keterangan'])
            writer.writerow(data_row)
    except Exception as e:
        print(f"⚠️ GAGAL mencatat riwayat: {e}")

✅ Folder output 'Kamus Mentah' telah disiapkan.


In [25]:
# Muat Data Korpus & Alat Bantu NLP

# Pastikan path benar
NAMA_FILE_KORPUS = 'Documents\corpus_master.csv' 
try:
    df_corpus = pd.read_csv(NAMA_FILE_KORPUS)
    print(f"✅ Berhasil memuat {len(df_corpus)} baris dari {NAMA_FILE_KORPUS}")
except FileNotFoundError:
    print(f"❌ ERROR: File korpus '{NAMA_FILE_KORPUS}' tidak ditemukan.")
    df_corpus = pd.DataFrame() # Buat dataframe kosong agar sel lain tidak error

# --- Siapkan Alat Bantu NLP (Copy dari notebook utama) ---
try:
    stopwords_id = set(stopwords.words('indonesian'))
    # Kustomisasi Stopwords (SANGAT PENTING)
    negation_words = ['tidak', 'kurang', 'jangan', 'bukan', 'tanpa']
    for word in negation_words:
        if word in stopwords_id:
            stopwords_id.remove(word)
    print("✅ Stopwords (termasuk kustomisasi negasi) siap.")
except:
    print("⚠️ Gagal memuat stopwords NLTK, gunakan daftar minimal.")
    stopwords_id = {"yang", "dan", "di", "ke", "ini"}

factory = StemmerFactory()
stemmer = factory.create_stemmer()
print("✅ Stemmer Sastrawi siap.")

#--- Membaca Kamus yang Sudah Ada ---
print("\n--- Membaca Kamus yang Sudah Ada ---")
existing_phrase_all = set()
existing_region_all = set()
existing_intent_all = set()

def load_existing_set(filepath, col1, col2, target_set):
    """Fungsi bantu untuk memuat Kolom A & B dari kamus ke 'target_set'."""
    try:
        df_kamus = pd.read_csv(filepath, dtype=str).fillna('')
        target_set.update(df_kamus[col1].str.lower())
        target_set.update(df_kamus[col2].str.lower())
        print(f"✅ Berhasil memuat {len(df_kamus)} aturan dari {filepath}")
    except FileNotFoundError:
        print(f"ℹ️ Info: File kamus {filepath} tidak ditemukan, akan menganggap kosong.")
    except Exception as e:
        print(f"❌ ERROR saat memuat {filepath}: {e}")

# Membaca semua kata di Kolom A (phrase) dan B (token) dari config_phrase_map
load_existing_set(
    os.path.join(FOLDER_KAMUS_UTAMA, 'config_phrase_map.csv'), 
    'Phrase', 'Token', existing_phrase_all
)
# Membaca semua kata di Kolom A (location_term) dan B (region_code) dari config_region_map
load_existing_set(
    os.path.join(FOLDER_KAMUS_UTAMA, 'config_region_map.csv'), 
    'location_term', 'region_code', existing_region_all
)
# Membaca semua kata di Kolom A (intent_phrase) dari config_special_intent
load_existing_set(
    os.path.join(FOLDER_KAMUS_UTAMA, 'config_special_intent.csv'),
    'intent_phrase', 'intent_code', existing_intent_all
)

print("\n✅ Daftar 'kata yang sudah ada' (existing sets) siap untuk memfilter.")

✅ Berhasil memuat 299 baris dari Documents\corpus_master.csv
✅ Stopwords (termasuk kustomisasi negasi) siap.
✅ Stemmer Sastrawi siap.

--- Membaca Kamus yang Sudah Ada ---
✅ Berhasil memuat 116 aturan dari Kamus\config_phrase_map.csv
✅ Berhasil memuat 32 aturan dari Kamus\config_region_map.csv
✅ Berhasil memuat 16 aturan dari Kamus\config_special_intent.csv

✅ Daftar 'kata yang sudah ada' (existing sets) siap untuk memfilter.


In [26]:
# Fungsi Bantu Preprocessing

# Fungsi ini diambil dari notebook utama (Cell 9)
def remove_special_characters(text):
    if not isinstance(text, str):
        return "" 
    regex = re.compile(r'[^a-zA-Z0-9\s]')
    return re.sub(regex, '', text)

# Fungsi ini adalah versi 'ringan' dari full_preprocessing
# Digunakan khusus untuk analisis n-gram
def preprocess_for_asisten(text):
    cleaned_text = remove_special_characters(text)
    cleaned_text = re.sub(r'\d', '', cleaned_text)
    words = cleaned_text.lower().split()
    words = [w for w in words if w not in stopwords_id]
    stemmed_words = [stemmer.stem(w) for w in words]
    return [w for w in stemmed_words if len(w) > 1]

print("✅ Fungsi bantu preprocessing siap.")

✅ Fungsi bantu preprocessing siap.


In [27]:
# ASISTEN 1 - Kandidat untuk 'config_region_map.csv'

print("============================================")
print("🤖 Asisten REGION_MAP Mulai...")
print("Menganalisis kolom 'Lokasi'...")
print("============================================")

try:
    lokasi_mentah = df_corpus['Lokasi'].dropna().unique()
    kandidat_region_unik = set()

    for lokasi in lokasi_mentah:
        # "Kab. Semarang, Jawa Tengah" -> ["Kab. Semarang", "Jawa Tengah"]
        parts = lokasi.split(',')
        for part in parts:
            cleaned_part = part.strip().lower() # .strip() hapus spasi, .lower() jadi huruf kecil
            if cleaned_part and len(cleaned_part) > 2: # Hindari "diy" jadi "d" atau string kosong
                kandidat_region_unik.add(cleaned_part)

    kandidat_region_baru = []
    for region in sorted(list(kandidat_region_unik)):
        # Hanya tambahkan jika region ini BELUM ADA di kamus region Anda
        if region not in existing_region_all:
            kandidat_region_baru.append(region)

    print("✅ SUKSES. Kandidat 'region_code' (Kolom B) yang ditemukan:")
    print("   (Salin ini ke Kolom B config_region_map.csv)")
    print("-" * 40)
    for region in sorted(list(kandidat_region_unik)):
        print(f"-> {region}")
    print("-" * 40)
    print("Tugas Anda: Isi Kolom A (location_term) dengan sinonim/variasinya (misal: 'jogja' untuk 'diy').")
    
    # Simpan ke CSV
    try:
        df_kandidat_region = pd.DataFrame(kandidat_region_baru, columns=['kandidat_region_baru'])
        NAMA_FILE_OUTPUT = os.path.join(FOLDER_KAMUS_MENTAH, 'kandidat_region.csv')
        df_kandidat_region.to_csv(NAMA_FILE_OUTPUT, index=False)
        print(f"\n💾 SUKSES: {len(kandidat_region_baru)} kandidat region BARU telah disimpan di '{NAMA_FILE_OUTPUT}'")
        
        log_event_ke_riwayat(
            kategori='Kandidat Region', 
            keterangan=f'Ditemukan {len(kandidat_region_baru)} kata baru. Daftar disimpan di {NAMA_FILE_OUTPUT}'
        )

    except Exception as e:
        print(f"\n❌ GAGAL menyimpan file kandidat region: {e}")

except KeyError:
    print("❌ ERROR: Kolom 'Lokasi' tidak ditemukan di korpus.")

🤖 Asisten REGION_MAP Mulai...
Menganalisis kolom 'Lokasi'...
✅ SUKSES. Kandidat 'region_code' (Kolom B) yang ditemukan:
   (Salin ini ke Kolom B config_region_map.csv)
----------------------------------------
-> banjarnegara
-> bantul
-> banyumas
-> diy
-> gunungkidul
-> jawa tengah
-> jepara
-> kab. semarang
-> karanganyar
-> kebumen
-> kendal
-> kudus
-> kulon progo
-> sleman
-> wonosobo
----------------------------------------
Tugas Anda: Isi Kolom A (location_term) dengan sinonim/variasinya (misal: 'jogja' untuk 'diy').

💾 SUKSES: 0 kandidat region BARU telah disimpan di 'Kamus Mentah\kandidat_region.csv'


In [28]:
# ASISTEN 2 - Kandidat untuk 'config_phrase_map.csv'

print("============================================")
print("🤖 Asisten PHRASE_MAP Mulai...")
print("Menganalisis 'Teks_Mentah' untuk frasa 2-kata (bigram) dan 3-kata (trigram)...")
print("(Ini mungkin butuh beberapa saat)")
print("============================================")

try:
    # Gabungkan SEMUA ulasan jadi satu teks raksasa
    semua_teks = " ".join(df_corpus['Teks_Mentah'].dropna().values)
    
    # Bersihkan teks (butuh waktu)
    all_tokens = preprocess_for_asisten(semua_teks)

    # --- Mencari Bigram (2-kata) ---
    print("\nTOP 20 Kandidat Bigram (2-kata):")
    print("   (Periksa ini untuk dimasukkan ke Kolom A config_phrase_map.csv)")
    print("-" * 40)
    # Menghitung frekuensi kemunculan setiap pasangan 2-kata
    bigram_freq = Counter(nltk.ngrams(all_tokens, 2))
    for (kata1, kata2), freq in bigram_freq.most_common(20):
        print(f"- ('{kata1}', '{kata2}') -> muncul {freq} kali")
    print("-" * 40)

    # --- Mencari Trigram (3-kata) ---
    print("\nTOP 20 Kandidat Trigram (3-kata):")
    print("   (Periksa ini untuk dimasukkan ke Kolom A config_phrase_map.csv)")
    print("-" * 40)
    # Menghitung frekuensi kemunculan setiap pasangan 3-kata
    trigram_freq = Counter(nltk.ngrams(all_tokens, 3))
    for (kata1, kata2, kata3), freq in trigram_freq.most_common(20):
         print(f"- ('{kata1}', '{kata2}', '{kata3}') -> muncul {freq} kali")
    print("-" * 40)
    print("Tugas Anda: Salin frasa yang bermakna (misal: 'kamar mandi', 'tidak bersih') ke Kolom A")
    print("            Lalu, TENTUKAN SENDIRI tokennya di Kolom B (misal: 'kamarmandi', 'tidakbersih').")

    # Simpan kandidat bigram ke CSV
    try:
        data_trigram = trigram_freq.most_common(100) # Simpan Top 100
        data_trigram_flat = [(k[0], k[1], k[2], v) for (k, v) in data_trigram]

        df_kandidat_trigram = pd.DataFrame(data_trigram_flat, columns=['kata_1', 'kata_2', 'kata_3', 'frekuensi'])
        NAMA_FILE_OUTPUT = os.path.join(FOLDER_KAMUS_MENTAH, 'kandidat_phrase.csv')
        df_kandidat_trigram.to_csv(NAMA_FILE_OUTPUT, index=False)
        print(f"\n💾 SUKSES: Top 100 kandidat Trigram telah disimpan di '{NAMA_FILE_OUTPUT}'")

        log_event_ke_riwayat(
            kategori='Kandidat Phrase', 
            keterangan=f'Ditemukan {len(data_trigram_flat)} kata baru. Daftar disimpan di {NAMA_FILE_OUTPUT}'
        )
        
    except Exception as e:
        print(f"\n❌ GAGAL menyimpan file kandidat trigram: {e}")

except KeyError:
    print("❌ ERROR: Kolom 'Teks_Mentah' tidak ditemukan di korpus.")

🤖 Asisten PHRASE_MAP Mulai...
Menganalisis 'Teks_Mentah' untuk frasa 2-kata (bigram) dan 3-kata (trigram)...
(Ini mungkin butuh beberapa saat)

TOP 20 Kandidat Bigram (2-kata):
   (Periksa ini untuk dimasukkan ke Kolom A config_phrase_map.csv)
----------------------------------------
- ('kamar', 'mandi') -> muncul 26 kali
- ('sewa', 'tenda') -> muncul 17 kali
- ('tiket', 'masuk') -> muncul 14 kali
- ('parkir', 'mobil') -> muncul 13 kali
- ('camping', 'ground') -> muncul 13 kali
- ('picnic', 'area') -> muncul 12 kali
- ('pandang', 'bagus') -> muncul 9 kali
- ('bukit', 'sikunir') -> muncul 9 kali
- ('bagus', 'banget') -> muncul 8 kali
- ('diri', 'tenda') -> muncul 8 kali
- ('fasilitas', 'lengkap') -> muncul 8 kali
- ('tempat', 'bagus') -> muncul 7 kali
- ('area', 'camp') -> muncul 7 kali
- ('sedia', 'toilet') -> muncul 6 kali
- ('pandang', 'indah') -> muncul 6 kali
- ('cocok', 'camping') -> muncul 6 kali
- ('akses', 'jalan') -> muncul 6 kali
- ('camp', 'ground') -> muncul 6 kali
- ('toil

In [29]:
# ASISTEN 3 - Kandidat untuk 'config_special_intent.csv'

print("============================================")
print("🤖 Asisten SPECIAL_INTENT Mulai...")
print("Menganalisis frasa berdasarkan 'Rating'...")
print("============================================")

try:
    # --- Ulasan Positif (Rating 4.5 ke atas) ---
    df_top = df_corpus[df_corpus['Rating'] >= 4.5]
    if not df_top.empty:
        print("\n⭐ TOP 15 Frasa dari Ulasan Rating Tinggi (>= 4.5)")
        print("   (Kandidat untuk intent 'RATING_TOP')")
        print("-" * 40)
        teks_top = " ".join(df_top['Teks_Mentah'].dropna().values)
        tokens_top = preprocess_for_asisten(teks_top)
        bigram_top = Counter(nltk.ngrams(tokens_top, 2))
        for (k1, k2), freq in bigram_top.most_common(15):
            print(f"- ('{k1}', '{k2}') -> muncul {freq} kali")
        print("-" * 40)

        # Simpan kandidat bigram top ke CSV
        try:
            data_top = bigram_top.most_common(50) # Simpan Top 50
            data_top_flat = [(k[0], k[1], v) for (k, v) in data_top]

            df_kandidat_top = pd.DataFrame(data_top_flat, columns=['kata_1', 'kata_2', 'frekuensi'])
            NAMA_FILE_OUTPUT = os.path.join(FOLDER_KAMUS_MENTAH, 'kandidat_intent_RATING_TOP.csv')
            df_kandidat_top.to_csv(NAMA_FILE_OUTPUT, index=False)
            print(f"\n💾 SUKSES: Top 50 kandidat RATING_TOP disimpan di '{NAMA_FILE_OUTPUT}'")

            log_event_ke_riwayat(
            kategori='Kandidat Rating Top', 
            keterangan=f'Ditemukan {len(data_top_flat)} kata baru. Daftar disimpan di {NAMA_FILE_OUTPUT}'
            )
            
        except Exception as e:
            print(f"\n❌ GAGAL menyimpan file kandidat RATING_TOP: {e}")

    else:
        print("Info: Tidak ditemukan ulasan dengan rating >= 4.5 untuk dianalisis.")

    # --- Ulasan Negatif (Rating 2.0 ke bawah) ---
    df_bottom = df_corpus[df_corpus['Rating'] <= 2.0]
    if not df_bottom.empty:
        print("\n👎 TOP 15 Frasa dari Ulasan Rating Rendah (<= 2.0)")
        print("   (Kandidat untuk intent 'RATING_BOTTOM')")
        print("-" * 40)
        teks_bottom = " ".join(df_bottom['Teks_Mentah'].dropna().values)
        tokens_bottom = preprocess_for_asisten(teks_bottom)
        bigram_bottom = Counter(nltk.ngrams(tokens_bottom, 2))
        for (k1, k2), freq in bigram_bottom.most_common(15):
            print(f"- ('{k1}', '{k2}') -> muncul {freq} kali")
        print("-" * 40)

        # Simpan kandidat bigram bottom ke CSV
        try:
            data_bottom = bigram_bottom.most_common(50) # Simpan Top 50
            data_bottom_flat = [(k[0], k[1], v) for (k, v) in data_bottom]

            df_kandidat_bottom = pd.DataFrame(data_bottom_flat, columns=['kata_1', 'kata_2', 'frekuensi'])
            NAMA_FILE_OUTPUT = os.path.join(FOLDER_KAMUS_MENTAH, 'kandidat_intent_RATING_BOTTOM.csv')
            df_kandidat_bottom.to_csv(NAMA_FILE_OUTPUT, index=False)
            print(f"\n💾 SUKSES: Top 50 kandidat RATING_BOTTOM disimpan di '{NAMA_FILE_OUTPUT}'")

            log_event_ke_riwayat(
            kategori='Kandidat Rating Bottom', 
            keterangan=f'Ditemukan {len(data_bottom_flat)} kata baru. Daftar disimpan di {NAMA_FILE_OUTPUT}'
            )
            
        except Exception as e:
            print(f"\n❌ GAGAL menyimpan file kandidat RATING_BOTTOM: {e}")

    else:
        print("Info: Tidak ditemukan ulasan dengan rating <= 2.0 untuk dianalisis.")

except KeyError:
    print("❌ ERROR: Kolom 'Teks_Mentah' atau 'Rating' tidak ditemukan di korpus.")

🤖 Asisten SPECIAL_INTENT Mulai...
Menganalisis frasa berdasarkan 'Rating'...

⭐ TOP 15 Frasa dari Ulasan Rating Tinggi (>= 4.5)
   (Kandidat untuk intent 'RATING_TOP')
----------------------------------------
- ('kamar', 'mandi') -> muncul 20 kali
- ('sewa', 'tenda') -> muncul 16 kali
- ('picnic', 'area') -> muncul 11 kali
- ('parkir', 'mobil') -> muncul 10 kali
- ('camping', 'ground') -> muncul 10 kali
- ('bukit', 'sikunir') -> muncul 9 kali
- ('tiket', 'masuk') -> muncul 8 kali
- ('fasilitas', 'lengkap') -> muncul 8 kali
- ('bagus', 'banget') -> muncul 7 kali
- ('diri', 'tenda') -> muncul 7 kali
- ('pandang', 'bagus') -> muncul 6 kali
- ('akses', 'jalan') -> muncul 6 kali
- ('toilet', 'mushola') -> muncul 6 kali
- ('area', 'camping') -> muncul 6 kali
- ('gunung', 'ungar') -> muncul 6 kali
----------------------------------------

💾 SUKSES: Top 50 kandidat RATING_TOP disimpan di 'Kamus Mentah\kandidat_intent_RATING_TOP.csv'

👎 TOP 15 Frasa dari Ulasan Rating Rendah (<= 2.0)
   (Kandid

In [30]:
# ASISTEN 4 - Kandidat BARU Kata Non-Baku (Slang, Singkatan, Typo)

print("============================================")
print("🤖 Asisten Kosakata Non-Baku Mulai...")
print("Menganalisis SEMUA token mentah untuk menemukan slang/singkatan...")
print("============================================")

try:
    semua_teks_mentah = " ".join(df_corpus['Teks_Mentah'].dropna().values)
    teks_bersih = remove_special_characters(semua_teks_mentah).lower()
    teks_bersih = re.sub(r'\d', '', teks_bersih)
    semua_token_mentah = teks_bersih.split()
    frekuensi_token = Counter(semua_token_mentah)
    
    print(f"Total token unik ditemukan (sebelum difilter): {len(frekuensi_token)}")
    
    MIN_FREQ = 5
    MAX_FREQ = 500
    
    print(f"\nMenyaring token dengan frekuensi antara {MIN_FREQ} s/d {MAX_FREQ} kali...")
    
    kandidat_non_baku_mentah = []
    for token, freq in frekuensi_token.items():
        if MIN_FREQ <= freq <= MAX_FREQ:
            kandidat_non_baku_mentah.append((token, freq))
            
    kandidat_non_baku_baru = []
    for token, freq in kandidat_non_baku_mentah:
        if token not in existing_phrase_all:
            kandidat_non_baku_baru.append((token, freq))
            
    kandidat_non_baku_baru.sort(key=lambda x: x[1], reverse=True)
    
    print(f"✅ SUKSES. Total kandidat di rentang 'sweet spot': {len(kandidat_non_baku_mentah)}")
    print(f"✅ Kandidat yang SUDAH ADA di kamus: {len(kandidat_non_baku_mentah) - len(kandidat_non_baku_baru)}")
    print(f"🔥 Kandidat BARU yang ditemukan: {len(kandidat_non_baku_baru)}")
    print("   (Hanya kandidat BARU yang akan dicetak dan disimpan)")
    print("-" * 40)

    try:
        df_kandidat_slang = pd.DataFrame(kandidat_non_baku_baru, columns=['kandidat_token_baru', 'frekuensi'])

        NAMA_FILE_OUTPUT = os.path.join(FOLDER_KAMUS_MENTAH, 'kandidat_NonBaku.csv')
        df_kandidat_slang.to_csv(NAMA_FILE_OUTPUT, index=False)
        print(f"\n💾 SUKSES: {len(kandidat_non_baku_baru)} kandidat slang BARU disimpan di '{NAMA_FILE_OUTPUT}'")
        
        log_event_ke_riwayat(
            kategori='Kandidat Slang', 
            keterangan=f'Ditemukan {len(kandidat_non_baku_baru)} kata baru. Daftar disimpan di {NAMA_FILE_OUTPUT}'
        )

    except Exception as e:
        print(f"\n❌ GAGAL menyimpan file kandidat slang: {e}")
    
    if not kandidat_non_baku_baru:
        print("Tidak ada kandidat slang/non-baku baru ditemukan. Kamus Anda sudah lengkap!")
    else:
        # Pindahkan print loop ke bawah
        for token, freq in kandidat_non_baku_baru:
            print(f"- '{token}' -> muncul {freq} kali")
            
    print("-" * 40)
    
except KeyError:
    print("❌ ERROR: Kolom 'Teks_Mentah' tidak ditemukan di korpus.")
except Exception as e:
    print(f"Terjadi error: {e}")

🤖 Asisten Kosakata Non-Baku Mulai...
Menganalisis SEMUA token mentah untuk menemukan slang/singkatan...
Total token unik ditemukan (sebelum difilter): 2316

Menyaring token dengan frekuensi antara 5 s/d 500 kali...
✅ SUKSES. Total kandidat di rentang 'sweet spot': 373
✅ Kandidat yang SUDAH ADA di kamus: 75
🔥 Kandidat BARU yang ditemukan: 298
   (Hanya kandidat BARU yang akan dicetak dan disimpan)
----------------------------------------

💾 SUKSES: 298 kandidat slang BARU disimpan di 'Kamus Mentah\kandidat_NonBaku.csv'
- 'dan' -> muncul 272 kali
- 'di' -> muncul 177 kali
- 'tenda' -> muncul 75 kali
- 'ke' -> muncul 74 kali
- 'buat' -> muncul 66 kali
- 'area' -> muncul 60 kali
- 'nya' -> muncul 59 kali
- 'banyak' -> muncul 51 kali
- 'parkir' -> muncul 50 kali
- 'bersih' -> muncul 48 kali
- 'saya' -> muncul 48 kali
- 'banget' -> muncul 47 kali
- 'ini' -> muncul 44 kali
- 'kesini' -> muncul 41 kali
- 'fasilitas' -> muncul 39 kali
- 'luas' -> muncul 39 kali
- 'lagi' -> muncul 38 kali
- 'tem